In [6]:
from transformers import AutoModelForImageTextToText, AutoProcessor
import torch

In [2]:
torch.cuda.is_available()

True

In [9]:
model_id = "Qwen/Qwen2.5-VL-3B-Instruct"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    device_map="auto",
    offload_buffers=True,
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [22]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Explain what overfitting is in one paragraph."}
        ]
    }
]

prompt = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True
)

In [23]:
prompt

[[151644,
  8948,
  198,
  2610,
  525,
  264,
  10950,
  17847,
  13,
  151645,
  198,
  151644,
  872,
  198,
  840,
  20772,
  1128,
  916,
  6276,
  1280,
  374,
  304,
  825,
  14311,
  13,
  151645,
  198,
  151644,
  77091,
  198]]

In [25]:
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=32,
        streamer=streamer
    )

result = processor.decode(output[0], skip_special_tokens=True)
print(result)

Overfitting occurs when a machine learning model learns the training data too well, to the point where it performs poorly on new, unseen data. This happens because
system
You are a helpful assistant.
user
Explain what overfitting is in one paragraph.
assistant
Overfitting occurs when a machine learning model learns the training data too well,


# Tools

In [27]:
def count_rows(dataset_path: str) -> int:
    return 11

In [28]:
TOOLS = {
    "count_rows": count_rows
}

In [29]:
SYSTEM_PROMPT = """
You are a data analysis assistant.

You have access to the following tools:

count_rows(dataset_path: string) -> int
  Use when the user asks about the number of rows in a CSV dataset.

Rules:
- If a tool is required, respond ONLY with valid JSON
- JSON format:
{
  "tool": "<tool_name>",
  "arguments": { ... }
}
- If no tool is needed, respond:
{
  "tool": null,
  "answer": "<final answer>"
}
- Do NOT explain your reasoning
- Do NOT add extra text
"""

In [46]:
import json

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": SYSTEM_PROMPT}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": "How many rows are in dataset.csv?"}]
    }
]

prompt = processor.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = processor(
    text=prompt,
    return_tensors="pt"
).to(model.device)


from transformers import TextStreamer

streamer = TextStreamer(
    processor.tokenizer,
    skip_prompt=True,
    skip_special_tokens=True
)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=128,
        streamer=streamer
    )

{
  "tool": "count_rows",
  "arguments": {
    "dataset_path": "dataset.csv"
  }
}


In [45]:
inputs['input_ids'].shape[-1]

137

In [54]:
prompt_len = inputs["input_ids"].shape[-1]

generated_tokens = output[0][prompt_len:]

result = processor.decode(
    generated_tokens,
    skip_special_tokens=True
)

print(result)

{
  "tool": "count_rows",
  "arguments": {
    "dataset_path": "dataset.csv"
  }
}


In [50]:
tool_call = json.loads(result)

tool_name = tool_call["tool"]
args = tool_call["arguments"]

result = TOOLS[tool_name](**args)
print("TOOL RESULT:", result)

TOOL RESULT: 11


In [56]:
type(result)

str